In [61]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Chrome

In [62]:
driver = Chrome()
driver.get("http://www.twitter.com/login")

In [65]:
username = driver.find_element_by_xpath('//input[@name="text"]')
user_name = input()
username.send_keys(user_name)
username.send_keys(Keys.RETURN)

/var/folders/z5/cmkn2nn57c5f79q4n4ndb6lc0000gn/T/ipykernel_1383/2384309533.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  username = driver.find_element_by_xpath('//input[@name="text"]')


In [66]:
password = driver.find_element_by_xpath('//input[@name="password"]')
my_password = getpass()
password.send_keys(my_password)
password.send_keys(Keys.RETURN)

/var/folders/z5/cmkn2nn57c5f79q4n4ndb6lc0000gn/T/ipykernel_1383/1819692316.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password = driver.find_element_by_xpath('//input[@name="password"]')


In [67]:
bookmarks_button = driver.find_element_by_xpath('//a[@aria-label="Bookmarks"]')
bookmarks_button.click()

/var/folders/z5/cmkn2nn57c5f79q4n4ndb6lc0000gn/T/ipykernel_1383/2979303500.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bookmarks_button = driver.find_element_by_xpath('//a[@aria-label="Bookmarks"]')


In [68]:
def get_tweet_data(card):
    """Extract data from tweet card"""
    user = card.find_element_by_xpath('.//span').text
    handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    # Sponsored posts don't return
    try:
        post_date = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    primary_content = card.find_element_by_xpath('./div/div/div/div[2]/div[2]/div[2]/div[1]').text
    secondary_content = card.find_element_by_xpath('./div/div/div/div[2]/div[2]/div[2]/div[2]').text
    tweet_body = primary_content + secondary_content
    tweet = (user, handle, post_date, tweet_body)

    return tweet

In [69]:
SCROLL_ATTEMPTS = 3
tweet_data = []
tweet_ids = set()
last_position = driver.execute_script('return window.pageYOffset;')
scrolling = True

while scrolling:
    cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    for card in cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            # Create a faux id
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                tweet_data.append(tweet)

    scroll_attempt = 0
    while True:
    # Scroll to the bottom
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(1)
        curr_position = driver.execute_script('return window.pageYOffset;')
        if last_position == curr_position:
            scroll_attempt += 1

            if scroll_attempt >= SCROLL_ATTEMPTS:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            last_position = curr_position
            break

/var/folders/z5/cmkn2nn57c5f79q4n4ndb6lc0000gn/T/ipykernel_1383/2760914625.py:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
/Users/femiojo/opt/anaconda3/envs/firstTestEnv/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


### Saving the data

In [71]:
with open('femiojo_BookmarksPt1.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['User Name', 'Handle', 'TimeStamp', 'Text']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(tweet_data)